# 鐵達尼號Kaggle實戰
參考自[XGBoost调参技巧（二）Titanic实战Top9%](https://zhuanlan.zhihu.com/p/28739256)

In [34]:
# -*- coding: utf-8 -*- 
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import svm
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
%matplotlib inline



## 讀取資料

In [35]:
train=pd.read_csv('train.csv',encoding='utf8')
test=pd.read_csv('test.csv',encoding='utf8')
submit = pd.read_csv('gender_submission.csv',encoding='utf8')

## data_cleasing_one_hot
這是使用one-hot encoding的方式來處理類別變數

## data_cleasing
這是單純把類別轉變為1、2、3的處理方式




In [36]:
def data_cleasing_one_hot(titanic):
    titanic["Age"] = titanic["Age"].fillna(titanic["Age"].median())
    # child
    titanic["child"] = titanic["Age"].apply(lambda x: 1 if x < 15 else 0)

    # sex
    titanic["sex"] = titanic["Sex"].apply(lambda x: 1 if x == "male" else 0)

    titanic["Embarked"] = titanic["Embarked"].fillna("S")
    #用眾數填入NA
    
    
    # familysize
    titanic["familysize"] = titanic["SibSp"] + titanic["Parch"] + 1

    titanic['Title1'] = titanic['Name'].str.split(", ", expand=True)[1]
    titanic['Title1'] = titanic['Title1'].str.split(".", expand=True)[0]
    titanic['Title2'] = titanic['Title1'].replace(['Mlle','Mme','Ms','Dr','Major','Lady','the Countess','Jonkheer','Col','Rev','Capt','Sir','Don','Dona'],
             ['Miss','Mrs','Miss','Mr','Mr','Mrs','Mrs','Mr','Mr','Mr','Mr','Mr','Mr','Mrs'])
    
    #將名字的謂稱提取出來
    
    
    titanic["Fare"] = titanic["Fare"].fillna(titanic["Fare"].median())
    
    one_hot=pd.get_dummies(titanic[['Embarked','Sex','Title2']])
    #將類別變數做one-hot encoding
    
    
    titanic=pd.concat([titanic,one_hot],axis=1)
    
    return titanic


def data_cleasing(titanic):
    titanic["Age"] = titanic["Age"].fillna(titanic["Age"].median())
    # child
    titanic["child"] = titanic["Age"].apply(lambda x: 1 if x < 15 else 0)

    # sex
    titanic["sex"] = titanic["Sex"].apply(lambda x: 1 if x == "male" else 0)

    titanic["Embarked"] = titanic["Embarked"].fillna("S")
    # embark
    def getEmbark(Embarked):
        if Embarked == "S":
            return 1
        elif Embarked == "C":
            return 2
        else:
            return 3
    titanic["embark"] = titanic["Embarked"].apply(getEmbark)

    # familysize
    titanic["fimalysize"] = titanic["SibSp"] + titanic["Parch"] + 1



    # name
    def getName(name):
        if "Mr" in str(name):
            return 1
        elif "Mrs" in str(name):
            return 2
        else:
            return 0
    titanic["name"] = titanic["Name"].apply(getName)
    
    titanic["Fare"] = titanic["Fare"].fillna(titanic["Fare"].median())

    return titanic

train_data = data_cleasing(train)
test_data  = data_cleasing(test)

In [37]:
features = ["Pclass", "sex", "child", "fimalysize", "Fare", "embark"]

In [41]:
train_set=train_data[features]

In [42]:
train_set['Pclass'] = train_set['Pclass'].astype('str')
train_set['sex'] = train_set['sex'].astype('str')
train_set['child'] = train_set['child'].astype('str')
train_set['embark'] = train_set['embark'].astype('str')


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/p

In [43]:
train_set = pd.get_dummies(train_set)

In [48]:
train_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
fimalysize    891 non-null int64
Fare          891 non-null float64
Pclass_1      891 non-null uint8
Pclass_2      891 non-null uint8
Pclass_3      891 non-null uint8
sex_0         891 non-null uint8
sex_1         891 non-null uint8
child_0       891 non-null uint8
child_1       891 non-null uint8
embark_1      891 non-null uint8
embark_2      891 non-null uint8
embark_3      891 non-null uint8
dtypes: float64(1), int64(1), uint8(10)
memory usage: 22.7 KB


In [128]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation

In [131]:
model = Sequential()
model.add(Dense(units=10, input_dim=12, kernel_initializer='normal', activation='relu')) 
model.add(Dense(units=8, activation='relu')) 
model.add(Dense(units=10, activation='relu')) 
model.add(Dense(units=16, activation='relu')) 
model.add(Dense(units=1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) 

In [132]:
train_history = model.fit(x=train_set, y=train_data['Survived'], validation_split=0.1, epochs=50, batch_size=40, verbose=2)  

Train on 801 samples, validate on 90 samples
Epoch 1/50
 - 1s - loss: 0.6895 - acc: 0.5680 - val_loss: 0.6698 - val_acc: 0.7556
Epoch 2/50
 - 0s - loss: 0.6685 - acc: 0.6779 - val_loss: 0.6444 - val_acc: 0.7333
Epoch 3/50
 - 0s - loss: 0.6560 - acc: 0.6792 - val_loss: 0.6270 - val_acc: 0.7444
Epoch 4/50
 - 0s - loss: 0.6436 - acc: 0.6792 - val_loss: 0.6091 - val_acc: 0.7667
Epoch 5/50
 - 0s - loss: 0.6204 - acc: 0.6991 - val_loss: 0.5849 - val_acc: 0.7444
Epoch 6/50
 - 0s - loss: 0.5869 - acc: 0.7091 - val_loss: 0.5598 - val_acc: 0.7444
Epoch 7/50
 - 0s - loss: 0.5616 - acc: 0.7328 - val_loss: 0.5317 - val_acc: 0.7889
Epoch 8/50
 - 0s - loss: 0.5464 - acc: 0.7478 - val_loss: 0.4955 - val_acc: 0.8000
Epoch 9/50
 - 0s - loss: 0.5198 - acc: 0.7790 - val_loss: 0.4725 - val_acc: 0.8000
Epoch 10/50
 - 0s - loss: 0.5088 - acc: 0.7890 - val_loss: 0.4542 - val_acc: 0.7778
Epoch 11/50
 - 0s - loss: 0.4960 - acc: 0.8002 - val_loss: 0.4584 - val_acc: 0.8111
Epoch 12/50
 - 0s - loss: 0.4844 - acc: 

In [134]:
test_set=test_data[features]
test_set['Pclass'] = test_set['Pclass'].astype('str')
test_set['sex'] = test_set['sex'].astype('str')
test_set['child'] = test_set['child'].astype('str')
test_set['embark'] = test_set['embark'].astype('str')
test_set=pd.get_dummies(test_set)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

In [136]:
y_pred=model.predict(test_set)

In [137]:
y_pred = np.where(y_pred>0.6,1,0)

In [138]:
test_data['Survived'] = y_pred

## 做成csv

In [139]:
submit=test_data[['PassengerId','Survived']]
submit['Survived'] = submit['Survived'].astype(int)
submit.to_csv('submit.csv', index= False)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
